In [18]:
import pandas as pd

In [19]:
#this class is merely for encapsulation.
#this is how data of the dataset is stores in the objects 
#of this class.
class DataSet:
    #name of the dataset
    def __init__(self, name):
        self.name = name
    
    def set_feature(self, features):
        self.features = features
    
    def set_classes(self, classes):
        self.classes = classes
    
    def get_classes(self):
        return self.classes
        
    def set_org_data(self, org_data):
        self.org_data = org_data
        
    def get_org_data(self):
        return self.org_data
    
    # not writing all getters, setters, because
    # they are not required in python (i didnt know that before)

In [20]:
#Class reads the data file and the names file
#to load inti dataset object
class Dataset_Reader:
    
    def __init__(self):
        self.Path = 'C:\\Users\\nitas\\Downloads\\Machine Learning 2024\\MLProject2\\'
    
    #read the data from data file and fill the Datset class object
    def read_data_file(self, ds):
        if(ds.name == 'forestfires'):
            df = pd.read_csv(self.Path+ds.name+'.data', delimiter=',')
        else:    
            df = pd.read_csv(self.Path+ds.name+'.data', delimiter=',', header=None)

        ds.set_org_data(df)
    
    
    #read name file, naming the features, just to fill
    #for results, not mendatory function
    def read_attributes(self, ds):
        
        start_string = 'Attribute Information'
        stop_string = 'Missing Attribute Values'
        isPrint = False
        
        #read section, where the file has information 
        #Attribute Information
        with open(Path+ds.name+'.names', 'r') as file:
            for line in file:
                if start_string in line:
                    isPrint = True
                    #continue
                    
                if stop_string in line:
                    isPrint = False
                    break
                        
                if isPrint:
                    print(line.strip())
                        

In [21]:
#Pre Process Class, for data cleaning, 
#checking for any abnormalities, pre processes the data 
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler


class PreProcess:
    
    def __init__(self, ds):
        
        #set other values of the Dataset object    
        self.set_Dataset_features(ds)
        
        #min max scaling for the regression datasets
        if(ds.type == 'Regression'):
            self.min_max_scaling(ds)
        if(ds.type == 'Classification'):
            self.min_max_scaling_class(ds)
        #now first step is to do one-hot encoding
        self.one_hot_encoding(ds)
        #Now sort the data, if it is not sorted
        self.sort_data(ds)
        #After this step, we will use only sorted data
        self.missing_values(ds)
    
    
    #Zscore Normalization 
    def zscore_normalization(self, ds):
        scaler = StandardScaler()
        numeric_cols = ds.org_data.select_dtypes(include=['float64', 'int64']).columns
        ds.org_data[numeric_cols] = scaler.fit_transform(ds.org_data[numeric_cols])
    
    
    def zscore_normalization_class(self, ds):
        scaler = StandardScaler()
        numeric_cols = ds.org_data.iloc[:, :-1].select_dtypes(include=['float64', 'int64']).columns
        ds.org_data[numeric_cols] = scaler.fit_transform(ds.org_data[numeric_cols])
        
    #Min max scaling 
    def min_max_scaling(self, ds):
        scaler = MinMaxScaler()
        #print(ds.org_data.head())
        # Apply scaling only to numeric columns
        numeric_cols = ds.org_data.select_dtypes(include=['float64', 'int64']).columns
        #print(numeric_cols)
        ds.org_data[numeric_cols] = scaler.fit_transform(ds.org_data[numeric_cols]) 
        #print('After applying Min Max Scaling:' ,ds.org_data.head())
        #Min max scaling 
        
    #for classification, we will exclude the class column    
    def min_max_scaling_class(self, ds):
        scaler = MinMaxScaler()
        #print(ds.org_data.head())
        # Apply scaling only to numeric columns
        numeric_cols = ds.org_data.iloc[:, :-1].select_dtypes(include=['float64', 'int64']).columns
        #print(numeric_cols)
        ds.org_data[numeric_cols] = scaler.fit_transform(ds.org_data[numeric_cols]) 
        #print('After applying Min Max Scaling:' ,ds.org_data.head())
    
    #This function sets the no of features and class names
    #to the Dataset class object
    def set_Dataset_features(self, ds):
        #set class names
        last_column_unique = ds.org_data.iloc[:, -1].unique()
        ds.set_classes(last_column_unique)
        #print(ds.get_classes())
        
        #set no of features, because one column is classes
        ds.feature_count = ds.org_data.shape[1] -1
        #print(ds.feature_count)
    
                
    #splitting data into 10 parts, Here we will
    #implement stratified 10 folds
    def split_data_regression(self,ds):
        
        # first we need to make 10 datasets of consectutive points
        #print('\n Printing the results of 10 folds')
        datagroupSize = len(ds.sorted_data) // 10  
        #print('Fold Size', datagroupSize)
        dataGroup = []
        
        #reusing this part from my first project
        for i in range(10):
            start = i * datagroupSize
            end = start + datagroupSize
            if i == 9:  
                dataGroup.append(ds.sorted_data.iloc[start:])
            else:
                dataGroup.append(ds.sorted_data.iloc[start:end])
        
        
        #once we have datasets of sorted data, now we will make folds
        #with one point from each fold.
        folds = [[] for _ in range(10)]
        
        max_group_size = max(len(group) for group in dataGroup)
        #print('Max Fold Size: ',max_group_size)
        
        # For each index up to the maximum group size
        for i in range(max_group_size):
            # Loop over the 10 groups
            for j in range(10):
                #print('fold:', j, 'datagroup :' ,j, 'value',  dataGroup[j].iloc[i, -1])
                # Check if the current group has enough data for the current index
                if i < len(dataGroup[j]):  
                    # Append the i-th example from the j-th group into the j-th fold
                    folds[j].append(dataGroup[j].iloc[i])

                    
        #converting folds into Dataframes
        final_folds = folds = [pd.DataFrame(fold) for fold in folds]
        ds.ten_folds = final_folds
        
        #for i in range(10):
        #    print(f'Fold {i + 1}:')
        #    print(final_folds[i].tail())
        
        #print("Ten Fold Prints")
        #for i, fold in enumerate(final_folds):
        #    print(f'Fold {i+1} size: {len(fold)}')
        #    print(fold.iloc[:, -1].value_counts())
    
    
    #splitting data into 10 parts, Here we will
    #implement stratified 10 folds for classification
    def split_data_classification(self,ds):
        
        #In this we first need to split data accordign to classes
        #Since we want the ratio of classes to be same in each fold
        classwise_data = {}
        
        for cls in ds.classes:
            classwise_data[cls] = ds.sorted_data[ds.sorted_data.iloc[:, -1] == cls]
                    
        # Now we will be creating the folds containing data from 
        # each fold of class, making the ratio same with original data
        folds = [[] for _ in range(10)]
        
        for cls, cls_data in classwise_data.items():
            
            #first divide the classwise data into 10 parts.
            clsPartsforFolds = len(cls_data) //10
            clsPartsRemain = len(cls_data) % 10
            
            #print('Number of class data per fold',clsPartsforFolds)
            #print('Remaining data left in class',clsPartsRemain)
            #put these values in all folds
            start = 0
            for i in range(10):
                if clsPartsRemain != 0:
                    # Add 1 to handle remaining data
                    end = start + clsPartsforFolds + 1  
                    clsPartsRemain -= 1
                else:
                    end = start + clsPartsforFolds
            
                # Append class data slice to the respective fold
                folds[i].extend(cls_data.iloc[start:end].values.tolist())
                start = end
            
        
        #converting folds into Dataframes
        final_folds = [pd.DataFrame(fold, columns=ds.sorted_data.columns) for fold in folds]    
        ds.ten_folds = final_folds
        
        #This section is just to check if the fold created are in the same ratio
        #for i in range(10):
        #    print(f'Fold {i + 1}:')
        #    print(final_folds[i].tail())
        
        #for i, fold in enumerate(final_folds):
        #    print(f'Fold {i+1} size: {len(fold)}')
        #    print(fold.iloc[:, -1].value_counts())
        
    
    #added the function to see, if we will need it for any dataset
    def one_hot_encoding(self, ds):
        #this has first coloumn of categorical data, F, M and I
        if(ds.name == 'abalone'):
            #save class column and drop it
            class_column = ds.org_data.iloc[:, -1]  
            ds.org_data = ds.org_data.drop(ds.org_data.columns[-1], axis=1)
            
            #perform one hot encoding    
            first_column_name = ds.org_data.columns[0]
            one_hot_encoded = pd.get_dummies(ds.org_data[first_column_name], prefix=first_column_name)
            ds.org_data = pd.concat([ds.org_data.drop(columns=[first_column_name]), one_hot_encoded], axis=1)
            #append the saved class column to the end again
            ds.org_data = pd.concat([ds.org_data, class_column], axis=1)
        
        #this dataset has first column of manufacturer, and second column of 
        #model name, which has unique values, so dropping it.
        if(ds.name == 'machine'):
            
            #here i am dropping the last colum, because class / result is 
            ds.org_data = ds.org_data.drop(ds.org_data.columns[-1], axis=1)
            #save the class column and then drop it   
            class_column = ds.org_data.iloc[:, -1]
            ds.org_data = ds.org_data.drop(ds.org_data.columns[-1], axis=1)
            
            #drop the second column, which is model number
            ds.org_data = ds.org_data.drop(ds.org_data.columns[1], axis=1)
            first_column_name = ds.org_data.columns[0]
            one_hot_encoded = pd.get_dummies(ds.org_data[first_column_name], prefix=first_column_name)
            ds.org_data = pd.concat([ds.org_data.drop(columns=[first_column_name]), one_hot_encoded], axis=1)
            
            #append the saved class column to the end again
            ds.org_data = pd.concat([ds.org_data, class_column], axis=1)
        
        #first column is months, second column is days
        if ds.name == 'forestfires':
            
            #tranform the last column to log 
            #*****This is still not helping, we might need normalization*****
            ds.org_data['area'] = np.log(ds.org_data['area'] +  1)
            
            
            #save the class column and then drop it, 
            class_column = ds.org_data.iloc[:, -1]
            ds.org_data = ds.org_data.drop(ds.org_data.columns[-1], axis=1)
            
            first_column_name = ds.org_data.columns[2]
            second_column_name = ds.org_data.columns[3]
            one_hot_encoded = pd.get_dummies(ds.org_data[[first_column_name, second_column_name]], 
                                             prefix=[first_column_name, second_column_name])
                                    
            ds.org_data = pd.concat([ds.org_data.drop(columns=[first_column_name, second_column_name]), 
                                     one_hot_encoded], axis=1)
            
            #append the saved class column to the end again
            ds.org_data = pd.concat([ds.org_data, class_column], axis=1)
        
        #print("One hot Encoding Step, Resulted data shown below")
        #print(ds.org_data.head())
        
    #First we need to sort the data, if not sorted according to the 
    #classes, if data is already sorted, then original values will remain
    #same without change
    def sort_data(self, ds):
        ds.sorted_data = ds.org_data.sort_values(by=ds.org_data.columns[-1])
        #and then reset the index
        ds.sorted_data = ds.sorted_data.reset_index(drop=True)
        
        
    # this function handles missing values, for this project
    # forest fires has no missing values
    # Abalone has no missing values
    # machine has no missing values
    # glass has no missing values
    # Soybean has no missing values
    # breastCancer has missing values
    def missing_values(self, ds):      
        # Check if any missing values exist, using '?' and null values
        ds.sorted_data.replace('?', np.nan, inplace=True)
        has_missing_values = ds.sorted_data.isnull().values.any()
        missing_values_count = ds.sorted_data.isnull().sum().sum()
        
        if (has_missing_values):
            #print("This dataset has missing values")
            if missing_values_count > 1:
                # It has missing values  
                #print("This dataset has more missing values")
                rows_with_missing = dds.sorted_data[ds.sorted_data.isnull().any(axis=1)].index
                #print("Rows with missing values before filling:")
                #print(ds.ds.sorted_data.loc[rows_with_missing])
                
                if(ds.name == 'breast-cancer-wisconsin'):
                    #print("Inside the median if")
                    for column in ds.sorted_data.columns[:-1]:
                        median_value = ds.sorted_data[column].median()
                        # Filling the missing values
                        ds.sorted_data[column].fillna(median_value, inplace=True)
    
                #print("Rows with missing values after filling:")
                #print(ds.sorted_data.loc[rows_with_missing])
            else:
                ds.sorted_data.dropna(inplace=True)
                print("Dropped the row")

    #This one function will work for classification    
    def split_tuning_data_classification(self, ds):
        #print('Spliting the data into tunning and remaining')
        # Backup sorted data
        ds.sorted_data_bk = ds.sorted_data.copy()
    
        # Initialize empty lists for tuning and remaining data
        tuning_data = []
        remaining_data = []

        # Separate the data into classes
        for cls in ds.classes:
            #print(cls)
            classwise_data = ds.sorted_data[ds.sorted_data.iloc[:, -1] == cls]

            # Shuffle the data
            classwise_data = classwise_data.sample(frac=1, random_state=42).reset_index(drop=True)
        
            # Extract 10% tuning set
            num_tuning_samples = max(1, int(len(classwise_data) * 0.1))  
            #print('number of samples from this class',num_tuning_samples )
        
            # Append tuning set and remaining data
            tuning_data.append(classwise_data.iloc[:num_tuning_samples])
            remaining_data.append(classwise_data.iloc[num_tuning_samples:])
    
        # Concatenate tuning and remaining data
        ds.tuning_data = pd.concat(tuning_data).reset_index(drop=True)
        ds.sorted_data = pd.concat(remaining_data).reset_index(drop=True)
        
        #print('Length of tuning data:',len(ds.tuning_data) )
        #print('Length of remaining data:',len(ds.sorted_data) )
        #print('Length of original data:',len(ds.sorted_data_bk) )

        
    def split_tuning_data_regression(self, ds):
        # Backup sorted data
        
        #print('\n Spliting the data into tunning and remaining')
        tuning_fraction = 0.1
        n_bins = 10
        ds.sorted_data_bk = ds.sorted_data.copy()

        # Bin the target variable into quantiles (or any number of bins)
        ds.sorted_data['bins'] = pd.qcut(ds.sorted_data.iloc[:, -1], q=n_bins, labels=False, duplicates='drop')

        # Initialize empty lists for tuning and remaining data
        tuning_data = []
        remaining_data = []

        # Separate the data into bins (which act as pseudo-classes for stratification)
        for bin_value in ds.sorted_data['bins'].unique():
            bin_data = ds.sorted_data[ds.sorted_data['bins'] == bin_value]

            # Shuffle the data
            bin_data = bin_data.sample(frac=1, random_state=42).reset_index(drop=True)

            # Extract the specified fraction (10% by default) for the tuning set
            num_tuning_samples = int(len(bin_data) * tuning_fraction)

            # Append tuning set and remaining data
            tuning_data.append(bin_data.iloc[:num_tuning_samples])
            remaining_data.append(bin_data.iloc[num_tuning_samples:])

        # Concatenate tuning and remaining data
        ds.tuning_data = pd.concat(tuning_data).reset_index(drop=True)
        ds.sorted_data = pd.concat(remaining_data).reset_index(drop=True)

        # Drop the 'bins' column, as it's not needed anymore
        ds.sorted_data.drop(columns=['bins'], inplace=True)
        ds.tuning_data.drop(columns=['bins'], inplace=True)

        # Check lengths
        #print('Length of tuning data:', len(ds.tuning_data))
        #print('Length of remaining data:', len(ds.sorted_data))
        #print('Length of original data:', len(ds.sorted_data_bk))    
        
        
        

In [22]:
class DifferentialEvolution:
    
    def __init__(self, population_size, mutation_factor, crossover_rate, generations):
        self.population_size = population_size
        self.mutation_factor = mutation_factor
        self.crossover_rate = crossover_rate
        self.generations = generations
        self.population = None

    def initialize_population(self, weight_dims, bounds=(-0.5, 0.5)):
        total_dim = sum((in_size * out_size) + out_size for in_size, out_size in weight_dims)
        self.population = np.random.uniform(bounds[0], bounds[1], (self.population_size, total_dim))

    
    
    def adjust_weights(self, nn, fitness_scores,x_data, y_data ):
                
        new_population = np.copy(self.population)

        for i in range(self.population_size):
            
            indices = np.random.choice(
                [idx for idx in range(self.population_size) if idx != i], 3, replace=False
            )
            x1, x2, x3 = self.population[indices]
            mutant = x1 + self.mutation_factor * (x2 - x3)

            # Crossover
            crossover_mask = np.random.rand(mutant.size) < self.crossover_rate
            trial = np.where(crossover_mask, mutant, self.population[i])

            nn.set_weights(trial)  
            trial_fitness = nn.calculate_loss(x_data, y_data)  

            # Selection
            if trial_fitness < fitness_scores[i]:
                new_population[i] = trial
                fitness_scores[i] = trial_fitness

        self.population = new_population
        return new_population[np.argmin(fitness_scores)]  


In [23]:
from abc import ABC, abstractmethod

class Layer(ABC): 
    def __init__(self):
        pass

    @abstractmethod
    def forward_propagation(self, inputData):
        pass


In [24]:
class FCHiddenLayer(Layer):
    
    def __init__(self, weights, bias, in_size, out_size):
        self.input_size = in_size
        self.output_size = out_size
        self.weights = weights
        self.bias = bias

        
    def forward_propagation(self, inputData):
        self.input = inputData  
        outputData = np.dot(self.input, self.weights) + self.bias  
        self.output = outputData
        return self.output
    
    
    def update_weights(self, weights, bias):
        self.weights = weights
        self.bias = bias


In [25]:
class ActivationLayer(Layer):
    
    def __init__(self, activation):
        self.activation = activation
        
    def forward_propagation(self, inputData):
        self.input = inputData  
        self.output = self.activation(self.input)  
        return self.output

    @staticmethod
    def tanh(x):
        return np.tanh(x)
    
    @staticmethod
    def sigmoid(x):
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))
            
    @staticmethod
    def linear(x):
        return x
    
    @staticmethod
    def softmax(x):
        exp_vals = np.exp(x - np.max(x))  
        return exp_vals / np.sum(exp_vals)


In [26]:
class NeuralNetwork:
    
    def __init__(self, lossFunction=None):
        self.layers = []
        self.loss = lossFunction
        
    def reset(self):
        self.layers = []

    def addLayers(self, layer):
        self.layers.append(layer)
    
    def set_weights(self, individual):
        idx = 0
       
        for layer in self.layers:
            if isinstance(layer, FCHiddenLayer):
                
                in_size, out_size = layer.input_size, layer.output_size
                weights, bias, idx = self.extract_weights_and_biases(individual, idx, in_size, out_size)
                layer.update_weights(weights, bias)

                
    def calculate_loss(self, train_features, train_class):
        return self.train(train_features, train_class)

    
    
    def extract_weights_and_biases(self,individual, idx, in_size, out_size):
        weight_size = in_size * out_size
        weights = individual[idx:idx + weight_size].reshape(in_size, out_size)
        idx += weight_size
        biases = individual[idx:idx + out_size]
        idx += out_size
        return weights, biases, idx
    
    
    def predict(self, test_features):
        rows = len(test_features)
        result = []

        for i in range(rows):
            output = test_features[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    def train(self, train_features, train_class, batch_size=32):
        no_of_samples = len(train_features)
        total_loss = 0  

        indices = np.arange(no_of_samples)
        np.random.shuffle(indices)
        x_train = train_features[indices]
        y_train = train_class[indices]

        for j in range(0, no_of_samples, batch_size):
            end = min(j + batch_size, no_of_samples)
            x_batch = x_train[j:end]
            y_batch = y_train[j:end]

            output = x_batch
            for layer in self.layers:
                output = layer.forward_propagation(output)

            batch_loss = self.loss(y_batch, output)
            total_loss += batch_loss  

            error = batch_loss  

        avg_loss = total_loss / (no_of_samples / batch_size)
        #print(f"Average Loss over all batches: {avg_loss}")
        return avg_loss
    
    
    @staticmethod
    def mse(actual_value, pred_value):
        return np.mean(np.power(actual_value-pred_value, 2))

    # for classification (binary)
    @staticmethod
    def binary_crossentropy(actual_value, pred_value):
        # Avoid log(0) by clipping values
        pred_value = np.clip(pred_value, 1e-12, 1 - 1e-12)
        return -np.mean(actual_value * np.log(pred_value) + (1 - actual_value) * np.log(1 - pred_value))


    # for multiclass classification
    @staticmethod
    def categorical_crossentropy(actual_value, pred_value):
        # Avoid log(0) by clipping values
        pred_value = np.clip(pred_value, 1e-12, 1 - 1e-12)
        return -np.mean(np.sum(actual_value * np.log(pred_value), axis=1))
    



In [27]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

class ClassificationNet:
    def __init__(self):
        self.net = None
    
    
    def tune_hyperparameters(self, ds, no_of_layers, input_size, x_train, y_train):
        
        generations_list = [50, 100, 150, 200, 300]
        mutation_factors = [0.1, 0.2, 0.3]
        crossover_rates = [0.1, 0.3, 0.5, 0.7, 0.9]

        best_accuracy = float('-inf')
        best_params = None
        best_weights = None

        
        for generations in generations_list:
            for mutation_factor in mutation_factors:
                for crossover_rate in crossover_rates:
                    print(f"Testing with generations={generations}, mutation_factor={mutation_factor}, crossover_rate={crossover_rate}")

                   
                    de_instance = DifferentialEvolution(
                        population_size=300, 
                        mutation_factor=mutation_factor, 
                        crossover_rate=crossover_rate, 
                        generations=generations
                    )

                    
                    if no_of_layers == 0:
                        layer_dims = [(input_size, 1)]
                    elif no_of_layers == 1:
                        layer_dims = [(input_size, 15), (15, 1)]
                    elif no_of_layers == 2:
                        layer_dims = [(input_size, 15), (15, 15), (15, 1)]

                    
                    de_instance.initialize_population(layer_dims)

                    for generation in range(de_instance.generations):
                        fitness_scores = []
                        for individual in de_instance.population:
                            idx = 0
                            self.net.reset()

                            
                            for in_size, out_size in layer_dims:
                                weights, biases, idx = self.extract_weights_and_biases(individual, idx, in_size, out_size)
                                self.net.addLayers(FCHiddenLayer(weights, biases, in_size, out_size))
                                activation_func = ActivationLayer.tanh if out_size != 1 else ActivationLayer.linear
                                self.net.addLayers(ActivationLayer(activation_func))

                            
                            error = self.net.train(x_train, y_train)
                            fitness_scores.append(error)

                        
                        best_weights = de_instance.adjust_weights(self.net, fitness_scores, x_train, y_train)

                        #print(f"Generation {generation}, Best Fitness: {min(fitness_scores)}")

                    accuracy = 1 - np.min(fitness_scores)  

                   
                    if accuracy > best_accuracy:
                        best_accuracy = accuracy
                        best_params = {'generations': generations, 'mutation_factor': mutation_factor, 'crossover_rate': crossover_rate}
                        best_weights = best_weights

        # Return the best parameters and weights
        print(f"Best Accuracy: {best_accuracy} with parameters: {best_params}")
        return best_params, best_weights
    
    
    
    
    # we initialize everything here
    def makeNeuralNetwork(self, ds, no_of_layers, input_size, x_train, y_train):
        
        if ds.name == 'breast-cancer-wisconsin':
            self.net = NeuralNetwork(NeuralNetwork.binary_crossentropy)
            output_activation = ActivationLayer.sigmoid
            output_size = 1
        else:
            self.net = NeuralNetwork(NeuralNetwork.categorical_crossentropy)
            output_activation = ActivationLayer.softmax
            output_size = len(ds.classes)
        
        # DE instance
        de_instance = DifferentialEvolution(population_size=250, mutation_factor=0.1, crossover_rate=0.8, generations=200)
        
        # Define network layer structure
        if no_of_layers == 0:
            layer_dims = [(input_size, output_size)]
        
        elif no_of_layers == 1:
            layer_dims = [(input_size, 16),(16, output_size)]

        elif no_of_layers == 2:
            layer_dims = [(input_size, 15),(15, 15),(15, output_size)]
        
    
        
        de_instance.initialize_population(layer_dims)

        for generation in range(de_instance.generations):
            fitness_scores = []
            for individual in de_instance.population:
                idx = 0
                self.net.reset()
                
                # Construct network layers from individual
                for in_size, out_size in layer_dims:
                    weights, biases, idx = self.extract_weights_and_biases(individual, idx, in_size, out_size)
                    self.net.addLayers(FCHiddenLayer(weights, biases, in_size, out_size))
                    activation_func = ActivationLayer.tanh if out_size != output_size else output_activation
                    self.net.addLayers(ActivationLayer(activation_func))

                # Calculate fitness (error)
                error = self.net.train(x_train, y_train)
                fitness_scores.append(error)

            # Update population based on fitness scores
            best_weights = de_instance.adjust_weights(self.net, fitness_scores, x_train, y_train )

            #print(f"Generation {generation}, Best Fitness: {min(fitness_scores)}")

        # Return the best weights after all generations
        return best_weights

    def extract_weights_and_biases(self, individual, idx, in_size, out_size):
        weight_size = in_size * out_size
        weights = individual[idx:idx + weight_size].reshape(in_size, out_size)
        idx += weight_size
        biases = individual[idx:idx + out_size]
        idx += out_size
        return weights, biases, idx
    
    
    
    def testFunction(self,ds, no_of_layers):
            
        ds.test_data = ds.ten_folds[9]
        ds.train_data = pd.concat([ds.ten_folds[i] for i in range(9)])

        train_features = ds.train_data.iloc[:, :-1]
        train_cls = ds.train_data.iloc[:, -1]
        test_features = ds.test_data.iloc[:, :-1]
        test_cls = ds.test_data.iloc[:, -1]

        x_train = train_features.to_numpy()
        y_train = train_cls.to_numpy()
        x_test = test_features.to_numpy()
        y_test = test_cls.to_numpy()
        input_size = train_features.shape[1]

        # One-Hot Encode the class labels
        encoder = OneHotEncoder(sparse_output=False)
        y_train = encoder.fit_transform(train_cls.values.reshape(-1, 1))
        y_test = encoder.transform(test_cls.values.reshape(-1, 1))

        # Train the neural network
        best_weights = self.makeNeuralNetwork(ds, no_of_layers, input_size, x_train, y_train)

        # Use the best weights to predict on the test set
        predictions = self.net.predict(x_test)
        predicted_classes = np.argmax(predictions, axis=1)
        true_classes = np.argmax(y_test, axis=1)

        # Compute accuracy for this fold
        accuracy = np.sum(predicted_classes == true_classes) / len(true_classes)
        print(f'Fold 1 Accuracy: {accuracy:.4f}')



    def mainFunction(self, ds, no_of_layers):
        fold_accuracies = []

        for i in range(len(ds.ten_folds)):  # Assuming len(ds.ten_folds) is 10 for 10-fold CV
            print(f'Fold: {i}')
            ds.test_data = ds.ten_folds[i]
            ds.train_data = pd.concat([ds.ten_folds[j] for j in range(len(ds.ten_folds)) if j != i])

            train_features = ds.train_data.iloc[:, :-1]
            train_cls = ds.train_data.iloc[:, -1]
            test_features = ds.test_data.iloc[:, :-1]
            test_cls = ds.test_data.iloc[:, -1]

            x_train = train_features.to_numpy()
            y_train = train_cls.to_numpy()
            x_test = test_features.to_numpy()
            y_test = test_cls.to_numpy()
            input_size = train_features.shape[1]

            # One-Hot Encode the class labels
            encoder = OneHotEncoder(sparse_output=False)
            y_train = encoder.fit_transform(train_cls.values.reshape(-1, 1))
            y_test = encoder.transform(test_cls.values.reshape(-1, 1))

            # Train the neural network
            best_weights = self.makeNeuralNetwork(ds, no_of_layers, input_size, x_train, y_train)

            # Use the best weights to predict on the test set
            predictions = self.net.predict(x_test)
            predicted_classes = np.argmax(predictions, axis=1)
            true_classes = np.argmax(y_test, axis=1)

            # Compute accuracy for this fold
            accuracy = np.sum(predicted_classes == true_classes) / len(true_classes)
            fold_accuracies.append(accuracy)
            print(f'Fold {i} Accuracy: {accuracy:.4f}')

        # Mean accuracy over all folds
        mean_accuracy = np.mean(fold_accuracies)
        print(f'Mean Accuracy: {mean_accuracy:.4f}')


In [28]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

class RegressionNet:
    def __init__(self):
        self.net = None
    
    
    # we initialize everything here
    def makeNeuralNetwork(self, ds, no_of_layers, input_size, x_train, y_train):
        
        self.net = NeuralNetwork(NeuralNetwork.mse)
        output_activation = ActivationLayer.linear
        output_size = 1
        
        # DE instance
        de_instance = DifferentialEvolution(population_size=250, mutation_factor=0.1, crossover_rate=0.8, generations=200)
        
        # Define network layer structure
        if no_of_layers == 0:
            layer_dims = [(input_size, output_size)]
        
        elif no_of_layers == 1:
            layer_dims = [(input_size, 17),(17, output_size)]

        elif no_of_layers == 2:
            layer_dims = [(input_size, 15),(15, 15),(15, output_size)]
        
    
        
        de_instance.initialize_population(layer_dims)

        for generation in range(de_instance.generations):
            fitness_scores = []
            for individual in de_instance.population:
                idx = 0
                self.net.reset()
                
                # Construct network layers from individual
                for in_size, out_size in layer_dims:
                    weights, biases, idx = self.extract_weights_and_biases(individual, idx, in_size, out_size)
                    self.net.addLayers(FCHiddenLayer(weights, biases, in_size, out_size))
                    activation_func = ActivationLayer.tanh if out_size != output_size else output_activation
                    self.net.addLayers(ActivationLayer(activation_func))

                # Calculate fitness (error)
                error = self.net.train(x_train, y_train)
                fitness_scores.append(error)

            # Update population based on fitness scores
            best_weights = de_instance.adjust_weights(self.net, fitness_scores, x_train, y_train )

            #print(f"Generation {generation}, Best Fitness: {min(fitness_scores)}")

        # Return the best weights after all generations
        return best_weights

    def extract_weights_and_biases(self, individual, idx, in_size, out_size):
        weight_size = in_size * out_size
        weights = individual[idx:idx + weight_size].reshape(in_size, out_size)
        idx += weight_size
        biases = individual[idx:idx + out_size]
        idx += out_size
        return weights, biases, idx

    
    def testFunction(self,ds, no_of_layers):

        
        ds.test_data = ds.ten_folds[9]
        ds.train_data = pd.concat([ds.ten_folds[i] for i in range(9)])

        train_features = ds.train_data.iloc[:, :-1]
        train_cls = ds.train_data.iloc[:, -1]
        test_features = ds.test_data.iloc[:, :-1]
        test_cls = ds.test_data.iloc[:, -1]

        x_train = train_features.to_numpy()
        y_train = train_cls.to_numpy()
        x_test = test_features.to_numpy()
        y_test = test_cls.to_numpy()
        input_size = train_features.shape[1]

        # Train the neural network
        best_weights = self.makeNeuralNetwork(ds, no_of_layers, input_size, x_train, y_train)

        predictions = self.net.predict(x_test)
        results = pd.DataFrame({'Actual Class': y_test, 'Predicted Class': predictions})

        mse = self.mse(results['Actual Class'], results['Predicted Class'])
        print(f'Fold 1: , Mean Squared Error: {mse}')

    
    def mainFunction(self, ds, no_of_layers):
        fold_errors = []

        for i in range(len(ds.ten_folds)):  # Assuming len(ds.ten_folds) is 10 for 10-fold CV
            print(f'Fold: {i}')
            ds.test_data = ds.ten_folds[i]
            ds.train_data = pd.concat([ds.ten_folds[j] for j in range(len(ds.ten_folds)) if j != i])

            train_features = ds.train_data.iloc[:, :-1]
            train_cls = ds.train_data.iloc[:, -1]
            test_features = ds.test_data.iloc[:, :-1]
            test_cls = ds.test_data.iloc[:, -1]

            x_train = train_features.to_numpy()
            y_train = train_cls.to_numpy()
            x_test = test_features.to_numpy()
            y_test = test_cls.to_numpy()
            input_size = train_features.shape[1]

            # Train the neural network
            best_weights = self.makeNeuralNetwork(ds, no_of_layers, input_size, x_train, y_train)

            predictions = self.net.predict(x_test)
            results = pd.DataFrame({'Actual Class': y_test, 'Predicted Class': predictions})

            mse = self.mse(results['Actual Class'], results['Predicted Class'])
            print(f'Fold: {i}, Mean Squared Error: {mse}')

            fold_errors.append(mse)

            print(f'Fold: {i}, Error: {mse}')

        # After all folds, calculate and print the mean error
        mean_errors = np.mean(fold_errors)
        print(f'Mean Error across all folds: {mean_errors}')
        
    def mse(self, actual_value, pred_value):
        return np.mean(np.power(actual_value - pred_value, 2))   


In [36]:
#Main driver class calls all other classes  

class Driver:
    def __init__(self):
        self.DSNames_classification = 'glass', 'soybean-small', 'breast-cancer-wisconsin'
        self.DSNames_regression = 'abalone', 'machine', 'forestfires'
    
    #classification
    def main_classification(self):
        
        #Preprocessing part
        #change the dataset number here from 0 to 2
        ds = DataSet(self.DSNames_classification[0])
        ds.type = 'Classification'
        ds_reader = Dataset_Reader()
        ds_reader.read_data_file(ds)
            
        pp = PreProcess(ds)
        pp.split_tuning_data_classification(ds)
        pp.split_data_classification(ds)
        
        cn = ClassificationNet()
        #print('Neural net with 0 hidden layers')
        #cn.mainFunction(ds, 0)
        #print('Neural net with 1 hidden layers')
        #cn.mainFunction(ds, 1)
        print('\nJust one fold result with 2 hidden layers for DE')
        cn.testFunction(ds,2)
       
        
    #Regression
    def main_regression(self):
        #change the dataset number here from 0 to 2
        ds = DataSet(self.DSNames_regression[1])
        ds.type = 'Regression'
        ds_reader = Dataset_Reader()
        ds_reader.read_data_file(ds)
        
        pp = PreProcess(ds)
        pp.split_tuning_data_regression(ds)
        pp.split_data_regression(ds)
        
        rn = RegressionNet()
        #print('Neural net with 0 hidden layers')
        #rn.mainFunction(ds, 0)
        #print('Neural net with 1 hidden layers')
        #rn.mainFunction(ds, 1)
        print('\nJust one fold result with 2 hidden layers for DE')
        rn.testFunction(ds,2)
        

In [37]:
#calling the driver class here.    
dr = Driver()
print('Classification Net')
dr.main_classification() 

Classification Net

Just one fold result with 2 hidden layers for DE
Fold 1 Accuracy: 0.5625

Just one fold result with 2 hidden layers for PSO
Fold 1 Accuracy: 0.8750

Just one fold result with 2 hidden layers for GA
Fold 1 Accuracy: 0.6875


In [38]:
print('Regression Net')
dr.main_regression()

Regression Net

Just one fold result with 2 hidden layers for DE
Fold 1: , Mean Squared Error: [0.18000208]

Just one fold result with 2 hidden layers for PSO
Fold 1: , Mean Squared Error: [0.17140149]

Just one fold result with 2 hidden layers for GA
Fold 1: , Mean Squared Error: [0.18240524]


In [32]:
#Main driver class calls all other classes  

class FullDriverClass:
    def __init__(self):
        self.DSNames_classification = 'glass', 'soybean-small', 'breast-cancer-wisconsin'
        self.DSNames_regression = 'abalone', 'machine', 'forestfires'


#classification
    def main_classification(self):
        
        #Preprocessing part
        #change the dataset number here from 0 to 2
        ds = DataSet(self.DSNames_classification[0])
        ds.type = 'Classification'
        ds_reader = Dataset_Reader()
        ds_reader.read_data_file(ds)
            
        pp = PreProcess(ds)
        pp.split_tuning_data_classification(ds)
        pp.split_data_classification(ds)
        
        cn = ClassificationNet()
        print('\n Neural net with 0 hidden layers, Average Accuracy')
        cn.mainFunction(ds, 0)
        print('\n Neural net with 1 hidden layers, Average Accuracy')
        cn.mainFunction(ds, 1)
        print('\n Neural net with 2 hidden layers, Average Accuracy')
        cn.mainFunction(ds, 2)
        
       
        
    #Regression
    def main_regression(self):
        #change the dataset number here from 0 to 2
        ds = DataSet(self.DSNames_regression[1])
        ds.type = 'Regression'
        ds_reader = Dataset_Reader()
        ds_reader.read_data_file(ds)
        
        pp = PreProcess(ds)
        pp.split_tuning_data_regression(ds)
        pp.split_data_regression(ds)
        
        rn = RegressionNet()
        print('\n Neural net with 0 hidden layers, Anerage MSE')
        rn.mainFunction(ds, 0)
        print('\n Neural net with 1 hidden layers, Average MSE')
        rn.mainFunction(ds, 1)
        print('\n Neural net with 2 hidden layers, Average MSE')
        rn.mainFunction(ds, 2)

        
fdr = FullDriverClass()
print('\n Classification Net for Glass Dataset')
fdr.main_classification() 
print('\n Regression Net for Machine Dataset')
fdr.main_regression()

        


 Classification Net for Glass Dataset

 Neural net with 0 hidden layers, Average Accuracy
Fold: 0
Fold 0 Accuracy: 0.6364
Fold: 1
Fold 1 Accuracy: 0.6818
Fold: 2
Fold 2 Accuracy: 0.6190
Fold: 3
Fold 3 Accuracy: 0.5500
Fold: 4
Fold 4 Accuracy: 0.6000
Fold: 5
Fold 5 Accuracy: 0.6500
Fold: 6
Fold 6 Accuracy: 0.6316
Fold: 7
Fold 7 Accuracy: 0.6667
Fold: 8
Fold 8 Accuracy: 0.6471
Fold: 9
Fold 9 Accuracy: 0.5625
Mean Accuracy: 0.6245

 Neural net with 1 hidden layers, Average Accuracy
Fold: 0
Fold 0 Accuracy: 0.7727
Fold: 1
Fold 1 Accuracy: 0.7727
Fold: 2
Fold 2 Accuracy: 0.7143
Fold: 3
Fold 3 Accuracy: 0.6500
Fold: 4
Fold 4 Accuracy: 0.8000
Fold: 5
Fold 5 Accuracy: 0.7500
Fold: 6
Fold 6 Accuracy: 0.6842
Fold: 7
Fold 7 Accuracy: 0.7222
Fold: 8
Fold 8 Accuracy: 0.8824
Fold: 9
Fold 9 Accuracy: 0.6875
Mean Accuracy: 0.7436

 Neural net with 2 hidden layers, Average Accuracy
Fold: 0
Fold 0 Accuracy: 0.5909
Fold: 1
Fold 1 Accuracy: 0.7273
Fold: 2
Fold 2 Accuracy: 0.1429
Fold: 3
Fold 3 Accuracy: 